# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorboard
import random as rand

# Load data

In [ ]:
data = pd.read_csv('sales_quantity.csv',names=['date','item_code','quantity'],header=0)
data.head()

In [ ]:
# Transform data

In [ ]:
#extract date features from date column
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_year'] = data['date'].dt.dayofyear



In [ ]:
#stack dataframe based on item_code
item_sales = data.groupby(['item_code','date','year','month','day','day_of_week','day_of_year'])['quantity'].sum().unstack(level=0)
#turn each NaN value to 0
item_sales = item_sales.fillna(0)

In [ ]:
item_sales

In [ ]:
#reset index and rename to 'index'
item_sales = item_sales.reset_index()
item_sales.columns.name = 'index'
item_sales.iloc[:,8:].head()

In [ ]:
#remove columns with sum below 50% quantile starting from 8th column
# high_sales = item_sales.loc[:,(item_sales.iloc[:,8:].sum()>item_sales.iloc[:,8:].sum().quantile(0.5))]
# high_sales
test = item_sales.iloc[:,8:].sum().map(lambda x: x > item_sales.iloc[:,8:].sum().quantile(0.5))
test

In [ ]:
#plot all items onto one graph
plt.figure()
plt.plot(item_sales.index,item_sales.iloc[:,7:], alpha=0.005, color = '0')
plt.show()